# Diagnostic Analysis of NHS Data Using Python

This notebook requires the `actual_duration.csv` and `appointments_regional.csv` and `national_categories.xlsx` files. Upload these files to the directory before you begin.

The csv files, Jupyter notebook and report can be accessed at the [GitHub repository][id2].

[id2]:https://github.com/lukebart/Bartholomew_Luke_DA201_Assignment

## 1: Describe The Data

Import and sense-check the data from the csv files using Pandas DataFrames. Determine (a) column names, number of rows and columns, data types and number of missing values, and (b) descriptive statistics and metadata of each DataFrame.

In [2]:
# Import packages with standard conventions
import numpy as np
import pandas as pd

In [10]:
# Function to describe data in the dataframe
# df = DataFrame object (dataframe)
# df_name = Name of the file the DataFrame object created from (string)
# df_columns = List of columns to describe unique values (list)
# df_sum = Column to sum (string)
# df_head = Show number of rows (integer)
def describe_data(df, df_name, df_columns, df_sum, df_head):
    print(f"Shape of DataFrame {df_name}:")
    print(df.shape)
    print('\n')
    print(f"Info of DataFrame {df_name}:")
    print(df.info())
    print('\n')
    print(f"Head of DataFrame {df_name}:")
    print(df.head())
    print('\n')
    print(f"Tail of DataFrame {df_name}:")
    print(df.tail())
    print('\n')
    print(f"Describe DataFrame {df_name}:")
    print(df.describe())
    print('\n')
    df_na = df[df.isna().any(axis=1)]  # missing values
    print(f"Missing values in DataFrame {df_name}:")
    print(df_na.shape)
    print('\n')
    # loop df_columns
    for col_name in df_columns:
        # Show unique values by col_name
        print(f"Unique count for {col_name} of DataFrame {df_name}: ")
        print(df[col_name].unique())
        print(df[col_name].nunique())
        print('\n')
        # Show sum of column df_sum by col_name
        print(f"Sum {df_sum} by {col_name} of DataFrame {df_name} (top {df_head}): ")
        print(df.groupby(col_name).aggregate({df_sum : 'sum'}).
              sort_values(by=[df_sum], ascending=False).head(df_head))
        print('\n')
    return


In [4]:
# Read the CSV files
ad = pd.read_csv('actual_duration.csv')
ar = pd.read_csv('appointments_regional.csv')

In [11]:
# Call function to describe data in actual_duration.csv
describe_data(ad,'actual_duration.csv',['sub_icb_location_ons_code',
                                        'icb_ons_code',
                                        'region_ons_code',
                                        'actual_duration'],
                                        'count_of_appointments',10)

Shape of DataFrame actual_duration.csv:
(137793, 8)


Info of DataFrame actual_duration.csv:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137793 entries, 0 to 137792
Data columns (total 8 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   sub_icb_location_code      137793 non-null  object
 1   sub_icb_location_ons_code  137793 non-null  object
 2   sub_icb_location_name      137793 non-null  object
 3   icb_ons_code               137793 non-null  object
 4   region_ons_code            137793 non-null  object
 5   appointment_date           137793 non-null  object
 6   actual_duration            137793 non-null  object
 7   count_of_appointments      137793 non-null  int64 
dtypes: int64(1), object(7)
memory usage: 8.4+ MB
None


Head of DataFrame actual_duration.csv:
  sub_icb_location_code sub_icb_location_ons_code  \
0                   00L                 E38000130   
1                   00L            

In [13]:
# Call function to describe data in appointments_regional.csv
describe_data(ar,'appointments_regional.csv',['icb_ons_code',
                                              'appointment_status', 
                                              'hcp_type', 
                                              'appointment_mode',
                                              'time_between_book_and_appointment'],
                                              'count_of_appointments',10)

Shape of DataFrame appointments_regional.csv:
(596821, 7)


Info of DataFrame appointments_regional.csv:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596821 entries, 0 to 596820
Data columns (total 7 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   icb_ons_code                       596821 non-null  object
 1   appointment_month                  596821 non-null  object
 2   appointment_status                 596821 non-null  object
 3   hcp_type                           596821 non-null  object
 4   appointment_mode                   596821 non-null  object
 5   time_between_book_and_appointment  596821 non-null  object
 6   count_of_appointments              596821 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 31.9+ MB
None


Head of DataFrame appointments_regional.csv:
  icb_ons_code appointment_month appointment_status hcp_type appointment_mode  \
0    E54000034           

In [13]:
# Read the Excel file
nc = pd.read_excel('national_categories.xlsx')

In [14]:
# Call function to describe data in national_categories.xlsx
describe_data(nc,'national_categories.xlsx',['sub_icb_location_name',
                                             'icb_ons_code',
                                             'service_setting', 
                                             'context_type', 
                                             'national_category'],
                                             'count_of_appointments',10)

Shape of DataFrame national_categories.xlsx:
(817394, 8)


Info of DataFrame national_categories.xlsx:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 817394 entries, 0 to 817393
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   appointment_date       817394 non-null  datetime64[ns]
 1   icb_ons_code           817394 non-null  object        
 2   sub_icb_location_name  817394 non-null  object        
 3   service_setting        817394 non-null  object        
 4   context_type           817394 non-null  object        
 5   national_category      817394 non-null  object        
 6   count_of_appointments  817394 non-null  int64         
 7   appointment_month      817394 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 49.9+ MB
None


Head of DataFrame national_categories.xlsx:
  appointment_date icb_ons_code                       sub_icb_loca

                                             count_of_appointments
sub_icb_location_name                                             
NHS North West London ICB - W2U3Z                         12142390
NHS North East London ICB - A3A8R                          9588891
NHS Kent and Medway ICB - 91Q                              9286167
NHS Hampshire and Isle Of Wight ICB - D9Y0V                8288102
NHS South East London ICB - 72Q                            7850170
NHS Devon ICB - 15N                                        7447758
NHS South West London ICB - 36L                            7155030
NHS Black Country ICB - D2P2L                              7033637
NHS North Central London ICB - 93C                         6747958
NHS Birmingham and Solihull ICB - 15E                      6383746


Unique count for icb_ons_code of DataFrame national_categories.xlsx: 
['E54000050' 'E54000048' 'E54000057' 'E54000008' 'E54000061' 'E54000060'
 'E54000054' 'E54000051' 'E54000015' 'E54000010' 'E

<span style="font-family:Helvetica">
<h2>Answer the Questions:</h2>
<br>
<b>Question 1. How many locations are there in the data set?</b>
   
<ul>
    <li>Sub-ICB Locations = 106</li>
    <li>ICB = 42</li>
    <li>Region = 7</li>
</ul>

> The number of locations by Sub-ICB, ICB & Region was cross-referenced 
    with official statistics on the UK Government [website][id1].

[id1]:https://geoportal.statistics.gov.uk/datasets/2bca16d4f8e4426d80137213fce90bbd_0/explore

<b>Question 2. What are the five locations with the highest number of records?</b>

> This was calculated as the sum of count_of_appointments by sub_icb_location_name from the dataset national_categories.xlsx. 

<table>
<thead>
    <tr><th>sub_icb_location_name</th><th>count_of_appointments</th></tr>
</thead>
<tbody>
    <tr><td>NHS North West London ICB - W2U3Z</td><td>12142390</td></tr>
    <tr><td>NHS North East London ICB - A3A8R</td><td>9588891</td></tr>
    <tr><td>NHS Kent and Medway ICB - 91Q</td><td>9286167</td></tr>
    <tr><td>NHS Hampshire and Isle Of Wight ICB - D9Y0V</td><td>8288102</td></tr>
    <tr><td>NHS South East London ICB - 72Q</td><td>7850170</td></tr>
</tbody>
</table>   

> <div class="alert alert-block alert-info">The reason to use the sum rather than count was that
it would be more useful to know the total sum of count_of_appointments by sub_icb_location_name rather 
    than only the number of records by sub_icb_location_name.</div>

                              
<b>Question 3. How many service settings, context types, national categories and appointment statuses are there?</b>

<table>
<thead>
    <tr><th>Category</th><th>Sub-categories</th><th>Count of Sub-categories</th></tr>
</thead>
<tbody>
    <tr><td>Service Settings</td><td>['Primary Care Network' 'Other'<br>'General Practice'<br>
      'Unmapped' 'Extended Access Provision']</td><td>5</td></tr>
    <tr><td>Context Types</td><td>['Care Related Encounter'<br>'Unmapped' 'Inconsistent Mapping']</td><td>3</td></tr>
    <tr><td>National Categories</td><td>['Patient contact during Care Home Round' 'Planned Clinics'<br>
        'Home Visit'
     'General Consultation Acute' 'Structured Medication Review'<br>
     'Care Home Visit' 'Unmapped' 'Clinical Triage'<br>
     'Planned Clinical Procedure' 'Inconsistent Mapping'<br>
     'Care Home Needs Assessment &amp Personalised Care and Support Planning'<br>
     'General Consultation Routine'<br>
     'Service provided by organisation external to the practice'<br>
     'Unplanned Clinical Activity' 'Social Prescribing Service'<br>
     'Non-contractual chargeable work'
     'Group Consultation and Group Education' 'Walk-in']</td><td>18</td></tr>
    <tr><td>Appointment Status</td><td>['Attended' 'DNA' 'Unknown']</td><td>3</td></tr>
</tbody>
</table>
    
> The categories above correspond correctly with the metadata provided with one exception - the "Inconsistant Mapping" and "Unmapped" sub-categories for National Categories does not exactly match the source metadata description text.
<hr>
